In [1]:
import os
#os.environ["OPENAI_API_KEY"]=""
os.environ["OPENAI_API_KEY"]=" insert your api key " 

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from duckduckgo_search import DDGS
from typing import List

# === Math Tool ===
class MathTool(BaseToolSpec):
    spec_functions = ["add", "sub", "mul", "div"]

    def add(self, a: int, b: int) -> int:
        """Add two numbers"""
        return a + b

    def sub(self, a: int, b: int) -> int:
        """Subtract two numbers"""
        return a - b

    def mul(self, a: int, b: int) -> int:
        """Multiply two numbers"""
        return a * b

    def div(self, a: int, b: int) -> float:
        """Divide two numbers"""
        return a / b

# === Document QA Tool ===
f_path = "D:/RAMANASOFT/alice-in-wonderland.pdf"
reader = SimpleDirectoryReader(input_files=[f_path])
pdf_documents = reader.load_data()
index = VectorStoreIndex.from_documents(pdf_documents)
query_engine = index.as_query_engine()

class DocumentTool(BaseToolSpec):
    spec_functions = ["ask_doc"]

    def ask_doc(self, question: str) -> str:
        """Answer questions based on Alice in Wonderland"""
        return str(query_engine.query(question))

# === DuckDuckGo Search Tool ===
class DuckDuckGoTool(BaseToolSpec):
    spec_functions = ["search_web"]

    def search_web(self, query: str) -> str:
        """Perform a web search using DuckDuckGo and return top 3 results"""
        results = []
        with DDGS() as ddgs:
            for r in ddgs.text(query, max_results=3):
                results.append(f"{r['title']}\n{r['href']}\n{r['body']}\n")
        return "\n---\n".join(results)

# === LLM and Agent Setup ===
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

tools = (
    MathTool().to_tool_list()
    + DocumentTool().to_tool_list()
    + DuckDuckGoTool().to_tool_list()
)

agent = OpenAIAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
)

# === Run Agent ===
if __name__ == "__main__":
    while True:
        query = input("Ask something: ")
        if query.lower() in ["exit", "quit"]:
            break
        response = agent.chat(query)
        print(f"\nResponse:\n{response}")


Ask something:  provide details about popular things in hyderabad india city


Added user message to memory: provide details about popular things in hyderabad india city
=== Calling Function ===
Calling function: search_web with args: {"query":"popular things in Hyderabad India"}
Got output: Top 27 Things to do in Hyderabad
https://www.hyderabadtourism.travel/things-to-do-in-hyderabad
#Trivia: The imposing gateway built with limestone and mortar commemorated Hyderabad's founding in 1591 and the new Islamic millennium (1000 AH). Palaces, mosques, hospitals, and active markets surround Charminar. One of the top things to do at Charminar is to go around shopping and come back with bags full of exciting souvenirs and trinkets. 4.

---
THE 15 BEST Things to Do in Hyderabad (2025) - Tripadvisor
https://www.tripadvisor.com/Attractions-g297586-Activities-Hyderabad_Hyderabad_District_Telangana.html
Things to Do in Hyderabad, India: See Tripadvisor's 246,887 traveler reviews and photos of Hyderabad tourist attractions. Find what to do today, this weekend, or in July. We ha

Ask something:  provide detail about openai


Added user message to memory: provide detail about openai
=== Calling Function ===
Calling function: search_web with args: {"query":"OpenAI"}
Got output: OpenAI
https://openai.com/
Economics and reasoning with OpenAI o1. ChatGPT Sep 13, 2024 2 min read. Latest research. View all. Toward understanding and preventing misalignment generalization. Publication Jun 18, 2025. Introducing HealthBench. Publication May 12, 2025. OpenAI for business. View all.

---
OpenAI Platform
https://platform.openai.com/
Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform.

---
OpenAI - Wikipedia
https://en.wikipedia.org/wiki/OpenAI
Former headquarters at the Pioneer Building in San Francisco. In December 2015, OpenAI was founded by Sam Altman, Elon Musk, Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk as the co-chairs. [16]

Ask something:  exit
